Add miniLM score for (generated_answer question, extracted_answer passage).

In [68]:
import json
from sentence_transformers import CrossEncoder

In [69]:
def read_rows(path):
    rows = []
    for line in open(path):
        rows.append(json.loads(line))
    return rows

def write_json_format(path_out, rows):
    f_out = open(path_out, 'w')
    for row in rows:
        f_out.write(json.dumps(row, ensure_ascii=False)+'\n')

In [70]:
path_dataset = 'test-B-big/legal.jl'
path_model = 'cross-encoder/ms-marco-MiniLM-L-6-v2'

In [71]:
rows = read_rows(path_dataset)

In [72]:
inputs_gpt3 = [(
    row['gpt3_answer'] + ' ' +row['question_translated'], 
    row['distillbert_answer'] + ' ' + row['passage_translated']) for row in rows]
inputs_chatgpt = [(
    row['chatgpt_answer'] + ' ' +row['question_translated'], 
    row['distillbert_answer'] + ' ' + row['passage_translated']) for row in rows]

In [73]:
model = CrossEncoder(path_model, max_length=512)

In [74]:
scores_gpt3 = model.predict(inputs_gpt3)
scores_chatgpt = model.predict(inputs_chatgpt)

In [75]:
for row, score_gpt3, score_chatgpt in zip(rows, scores_gpt3, scores_chatgpt):
    row['score_miniLM_with_gpt3'] = float(score_gpt3)
    row['score_miniLM_with_chatgpt'] = float(score_chatgpt)

In [76]:
write_json_format(path_dataset, rows)